In [1]:

%pprint
import sys
sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [2]:

from notebook_utils import NotebookUtilities
import os
import os.path as osp
import re

nu = NotebookUtilities(
    data_folder_path=osp.abspath('../data'),
    saves_folder_path=osp.abspath('../saves')
)


# Search for Scenario and Probe Vocabulary

In [3]:

# Load the NER entities from a CSV
if nu.csv_exists('domain_doc_ners_df'):
    domain_doc_ners_df = nu.load_data_frames(domain_doc_ners_df='domain_doc_ners_df')['domain_doc_ners_df']
    print(domain_doc_ners_df.columns.tolist())

No pickle exists for domain_doc_ners_df - attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/domain_doc_ners_df.csv.
['bert_word', 'bert_entity', 'bert_score', 'file_path', 'nlp_word', 'nlp_tag', 'nlp_type', 'nlp_pofs', 'ent_phrase', 'ent_type', 'ent_start', 'ent_end']


In [4]:

# Iterate through each NER column and display a sample dataset where the column has values
for cn in domain_doc_ners_df.columns:
    mask_series = domain_doc_ners_df[cn].isnull()
    df = domain_doc_ners_df[~mask_series]
    if df.shape[0]:
        print(cn)
        display(df.sample(min(4, df.shape[0])).dropna(axis='columns', how='all').T)

bert_word


,1014216,767406,43999,285321
bert_word,University,Greek,Defense,Research
bert_entity,I-ORG,I-MISC,I-ORG,I-ORG
bert_score,0.799285,0.996943,0.998056,0.996991
file_path,../data/Domain_Knowledge/Military Medical Ethi...,../data/Domain_Knowledge/Military Medical Ethi...,../data/Domain_Knowledge/Tactical_Combat_Casua...,../data/Domain_Knowledge/Fundamentals of Milit...


bert_entity


,511866,335364,954966,1297679
bert_word,Beth,of,Sc,Road
bert_entity,I-LOC,I-ORG,I-ORG,I-LOC
bert_score,0.995348,0.966635,0.655862,0.913749
file_path,../data/Domain_Knowledge/Fundamentals of Milit...,../data/Domain_Knowledge/Fundamentals of Milit...,../data/Domain_Knowledge/Military Medical Ethi...,../data/Domain_Knowledge/Military Medical Ethi...


bert_score


...

,38041,284843,1012237,1488640
file_path,../data/Domain_Knowledge/Prolonged_Casualty_Ca...,../data/Domain_Knowledge/Fundamentals of Milit...,../data/Domain_Knowledge/Military Medical Ethi...,../data/Domain_Knowledge/Rules of Engagement/A...
ent_phrase,13 Jan 2017,0,Shanghai,GPW
ent_type,DATE,CARDINAL,GPE,ORG
ent_start,139043.0,16607.0,87855.0,363914.0
ent_end,139054.0,16608.0,87863.0,363917.0


ent_start


,301087,1275728,131692,864107
file_path,../data/Domain_Knowledge/Fundamentals of Milit...,../data/Domain_Knowledge/Military Medical Ethi...,../data/Domain_Knowledge/Fundamentals of Milit...,../data/Domain_Knowledge/Military Medical Ethi...
ent_phrase,134,"San Miguel,33",Department of Defense,US
ent_type,CARDINAL,GPE,ORG,GPE
ent_start,34459.0,90149.0,33367.0,105894.0
ent_end,34462.0,90162.0,33388.0,105896.0


ent_end


,952868,1012780,952606,38612
file_path,../data/Domain_Knowledge/Military Medical Ethi...,../data/Domain_Knowledge/Military Medical Ethi...,../data/Domain_Knowledge/Military Medical Ethi...,../data/Domain_Knowledge/Prolonged_Casualty_Ca...
ent_phrase,Nazi,Hiroshima,Germany,11
ent_type,NORP,GPE,GPE,CARDINAL
ent_start,19376.0,122859.0,4351.0,175157.0
ent_end,19380.0,122868.0,4358.0,175159.0


In [5]:

# Attempt to load the logs data frame from a pickle
if nu.pickle_exists('frvrs_logs_df'):
    try:
        frvrs_logs_df = nu.load_object('frvrs_logs_df')
        print(frvrs_logs_df.shape)
    except Exception as e:
        print(str(e).strip())
        
        # Load data frame from CSV
        if nu.csv_exists('frvrs_logs_df'):
            frvrs_logs_df = nu.load_data_frames(frvrs_logs_df='frvrs_logs_df')['frvrs_logs_df']
            print(frvrs_logs_df.shape)

Argument 'placement' has incorrect type (expected pandas._libs.internals.BlockPlacement, got slice)
Attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/frvrs_logs_df.pkl.
Argument 'placement' has incorrect type (expected pandas._libs.internals.BlockPlacement, got slice)
No pickle exists for frvrs_logs_df - attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/frvrs_logs_df.csv.
(829116, 114)


In [37]:

# Create a list of starting phrases from the tool and procedure columns
# of the logs dataset to search for in the NER entities dataset to label
# for model training
words_list = []
for cn in ['tool_hover_type', 'injury_record_required_procedure', 'injury_treated_required_procedure', 'tool_applied_data']:

    # Get unique lowercase words from the column and split into a list
    words_list.extend(' '.join(frvrs_logs_df[cn].map(lambda x: str(x).lower()).unique().tolist()).split(' '))

# Create a set of unique relevant words that have been cleaned for improved matching
canonical_phrases = [
    word_str.replace('(clone)', '').replace('_', ' ').replace('pressure', ' pressure').replace('pack', ' pack')
    for word_str in list(set(words_list) - set(['nan', 'none']))
]

# Find columns containing any of the canonical phrases using regular expressions
regexed_columns_list = nu.get_regexed_columns(domain_doc_ners_df, search_regex=re.compile('.*(' + '|'.join(canonical_phrases) + ').*', re.IGNORECASE))

# Iterate through each found column and each canonical phrase and
# display a sample dataset where the phrase exists in the column's values
word_columns_list = [
    'bert_word', 'nlp_word', 'ent_phrase',
    'bert_entity', 'nlp_tag', 'nlp_type', 'nlp_pofs', 'ent_type'
]
for cn in set(regexed_columns_list):
    for word_str in set(canonical_phrases):
        mask_series = domain_doc_ners_df[cn].map(lambda x: word_str in str(x).lower())
        df = domain_doc_ners_df[mask_series][word_columns_list]
        if df.shape[0]:
            print(f'Column: {cn}, Phrase: {word_str}')
            display(df.sample(min(10, df.shape[0])).dropna(axis='columns', how='all').T)

# Print a message about potential model training
nouns_str = nu.conjunctify_nouns([f'"{word_str}"' for word_str in set(canonical_phrases) if not ('(' in word_str)])
print(f'I can train a CRF or LR model to predict on entities that contain phrases like {nouns_str}.')

Column: ent_phrase, Phrase: tourniquet


,83333,83326,83352,83289,83331,82998,83877,385382,82925,384696
ent_phrase,Emergency Medical Tourniquet,Combat Application Tourniquet,JUNCTIONAL TOURNIQUETS & DEVICES,Tourniquet/Time Applied\nDrugs,Tourniquet,Tourniquet,SAM-Junctional Tourniquet,Keenan S. Tourniquet,Limb Tourniquet,the SAM Junctional Tourniquet
ent_type,ORG,ORG,ORG,ORG,ORG,ORG,ORG,PERSON,PERSON,ORG


Column: ent_phrase, Phrase: hemostatic


,62555,54257,83344,83751,384608
ent_phrase,• Hemostatic,• Hemostatic,Hemostatic Celox Gauze,CHITOSAN-BASED HEMOSTATIC GAUZE \nDRESSINGS: T...,Hemostatic Dressings\nThe
ent_type,ORG,ORG,PERSON,ORG,ORG


Column: ent_phrase, Phrase: gauze


...

Column: nlp_word, Phrase: needle


,48446,806036,52453,48231,806952,56384,48128,142640,40647,1213599
nlp_word,needle,needless,needle,needle,needless,needle,needle,needle,needle,needles
nlp_tag,NN,JJ,NN,NN,JJ,NN,NN,NN,NN,NNS
nlp_pofs,NOUN,ADJ,NOUN,NOUN,ADJ,NOUN,NOUN,NOUN,NOUN,NOUN


Column: nlp_word, Phrase: decompress


,42497,359040,74120,27764,233438,48129,48470,67998,243845,246711
nlp_word,decompression,decompression,decompression,decompression,decompression,decompression,decompression,Decompress,decompression,decompression
nlp_tag,NN,NN,NN,NN,NN,NN,NN,NNP,NN,NN
nlp_pofs,NOUN,NOUN,NOUN,NOUN,NOUN,NOUN,NOUN,PROPN,NOUN,NOUN


I can train a CRF or LR model to predict on entities that contain phrases like "left chest", "tourniquet", "hemostatic", "gauze pressure", "gauze", "right chest", "airway", "nasal", "left underarm", "right underarm", "needle", "decompress", "wound pack", and "naso airway".


In [217]:

from pandas import DataFrame, Series

rows_list = []
indices_list = []
cns_list = []
word_strs_list = []
if 'is_probe' not in domain_doc_ners_df.columns: domain_doc_ners_df['is_probe'] = False
for cn in set(regexed_columns_list):
    for word_str in set(canonical_phrases):
        mask_series = domain_doc_ners_df[cn].map(lambda x: word_str in str(x).lower())
        df = domain_doc_ners_df[mask_series][word_columns_list].dropna(axis='columns', how='all')
        if df.shape[0]:
            # domain_doc_ners_df.loc[mask_series, 'is_probe'] = True
            rows_list.extend([v for k, v in df.T.to_dict().items()])
            indices_list.extend(df.index.tolist())
            cns_list.extend([cn]*df.shape[0])
            word_strs_list.extend([word_str]*df.shape[0])
isprobe_df = DataFrame(rows_list)
isprobe_df['entity_index'] = Series(indices_list)
isprobe_df['entity_column_name'] = Series(cns_list)
isprobe_df['entity_word_str'] = Series(word_strs_list)
isprobe_df['is_probe'] = True
string_to_integer_dict_dict = {}
for cn in ['ent_type', 'nlp_tag', 'nlp_type', 'nlp_pofs', 'entity_column_name', 'entity_word_str']:
    integer_array, string_to_integer_dict = nu.convert_strings_to_integers(isprobe_df[cn])
    isprobe_df[cn] = Series(integer_array)
    string_to_integer_dict_dict[cn] = string_to_integer_dict
isprobe_df.columns.tolist()
isprobe_df

,ent_phrase,ent_type,nlp_word,nlp_tag,nlp_type,nlp_pofs,entity_index,entity_column_name,entity_word_str,is_probe
0,Limb Tourniquet,2,NaN,0,0,0,82598,1,0,True
1,2nd Tourniquet,1,NaN,0,0,0,82599,1,0,True
2,Tourniquet,4,NaN,0,0,0,82670,1,0,True
3,Limb Tourniquet,2,NaN,0,0,0,82925,1,0,True
4,2nd Tourniquet,1,NaN,0,0,0,82926,1,0,True
...,...,...,...,...,...,...,...,...,...,...
963,NaN,0,decompression,7,0,1,467425,0,6,True
964,NaN,0,decompression,7,0,1,478598,0,6,True
965,NaN,0,decompression,7,0,1,479066,0,6,True
966,NaN,0,decompression,7,0,1,938327,0,6,True



## Clean up ent_phrase column

In [204]:

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier

vectorizer = CountVectorizer(
    lowercase=True, ngram_range=(1, 3)
)
tfidf_transformer = TfidfTransformer(
    norm='l1', smooth_idf=True, sublinear_tf=False, use_idf=True
)
classifier = SGDClassifier(loss='log_loss', warm_start=True)
mask_series = domain_doc_ners_df.ent_phrase.isnull()
columns_list = ['ent_phrase', 'is_probe']
df = domain_doc_ners_df[~mask_series][columns_list]
df.is_probe = df.is_probe.map(
    lambda x: {True: 1, False: 0}.get(x, x)
)
train_data_list = df.ent_phrase.tolist()
train_labels_list = df.is_probe.values
X_train_counts = vectorizer.fit_transform(train_data_list)
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

# Train on initial data
classifier.fit(X_train_tfidf, train_labels_list)

SGDClassifier(loss='log_loss', warm_start=True)

In [212]:

mask_series = domain_doc_ners_df.ent_phrase.isnull()
sample_ent_phrase = domain_doc_ners_df[~mask_series].sample(1).ent_phrase.squeeze()
display(sample_ent_phrase)
X_test = tfidf_transformer.transform(vectorizer.transform([sample_ent_phrase])).toarray()
display(classifier.predict_proba(X_test)[0][1])

'the Laws of War'

0.002734841657490865

In [213]:

if 'is_probe_probability' not in domain_doc_ners_df.columns: domain_doc_ners_df['is_probe_probability'] = 0.0
mask_series = domain_doc_ners_df.ent_phrase.isnull()
domain_doc_ners_df.loc[~mask_series, 'is_probe_probability'] = domain_doc_ners_df[~mask_series].ent_phrase.map(
    lambda x: classifier.predict_proba(tfidf_transformer.transform(vectorizer.transform([x])).toarray())[0][1]
)

In [216]:

mask_series = ~domain_doc_ners_df.is_probe & (domain_doc_ners_df.is_probe_probability > 0.0)
analysis_columns = [
    'bert_word', 'bert_entity', 'bert_score', 'nlp_word', 'nlp_tag', 'nlp_type', 'nlp_pofs', 'ent_phrase', 'ent_type',
    'is_probe', 'is_probe_probability'
]
print(domain_doc_ners_df.columns.tolist())
display(domain_doc_ners_df[mask_series][analysis_columns].sort_values('is_probe_probability', ascending=False).head(60).tail(20).dropna(axis='columns', how='all'))

['bert_word', 'bert_entity', 'bert_score', 'file_path', 'nlp_word', 'nlp_tag', 'nlp_type', 'nlp_pofs', 'ent_phrase', 'ent_type', 'ent_start', 'ent_end', 'is_probe', 'is_probe_probability']


,ent_phrase,ent_type,is_probe,is_probe_probability
745354,Combat Ethics,ORG,False,0.003861
746000,Combat Ethics,ORG,False,0.003861
745261,COMBAT ETHICS,ORG,False,0.003861
747425,Combat Ethics,ORG,False,0.003861
745298,Combat Ethics,ORG,False,0.003861
746234,Combat Ethics,ORG,False,0.003861
809555,Combat Ethics,ORG,False,0.003861
745264,COMBAT ETHICS,ORG,False,0.003861
747116,Combat Ethics,ORG,False,0.003861
276306,Combat Duty,ORG,False,0.003642



### ent_phrase Maintenance

In [140]:

# test_words_list = list(set(re.split(r'\s+', ' '.join([str(x) for x in domain_doc_ners_df[domain_doc_ners_df.is_probe].ent_phrase.tolist()]))))
test_words_list = [
    'chest', 'chin', 'chito', 'compression', 'cpg', 'decompress', 'extraction', 'gauze', 'glassberg', 'hemcon', 'hemorrhage', 'hemostatic',
    'jaw', 'junctional', 'kaolin', 'keenan', 'lifesaver', 'medical', 'nasopharyngeal', 'needle', 'obstruction', 'pcc'
]
print(sorted(set(test_words_list)))

['chest', 'chin', 'chito', 'compression', 'cpg', 'decompress', 'extraction', 'gauze', 'glassberg', 'hemcon', 'hemorrhage', 'hemostatic', 'jaw', 'junctional', 'kaolin', 'keenan', 'lifesaver', 'medical', 'nasopharyngeal', 'needle', 'obstruction', 'pcc']


In [203]:

word_str = test_words_list.pop()
mask_series = domain_doc_ners_df.ent_phrase.map(lambda x: word_str.lower() in str(x).lower()) & domain_doc_ners_df.is_probe
df = domain_doc_ners_df[mask_series][word_analysis_columns]
if df.shape[0]:
    display(word_str)
    display(df.dropna(axis='columns', how='all'))

IndexError: pop from empty list

In [ ]:

mask_series = domain_doc_ners_df.ent_phrase.map(lambda x: 'Xxxxxx' in str(x).lower())# & (domain_doc_ners_df.ent_type == 'PERSON')
# domain_doc_ners_df.loc[mask_series, 'is_probe'] = True
# nu.save_data_frames(domain_doc_ners_df=domain_doc_ners_df)
display(domain_doc_ners_df[mask_series][analysis_columns].sort_values('is_probe_probability', ascending=False).dropna(axis='columns', how='all'))

In [201]:

mask_series = domain_doc_ners_df.ent_phrase.map(lambda x: str(x).startswith('• Chin'))
domain_doc_ners_df.loc[mask_series, 'is_probe'] = True
nu.save_data_frames(domain_doc_ners_df=domain_doc_ners_df)
display(domain_doc_ners_df[mask_series][analysis_columns].sort_values('is_probe_probability', ascending=False).dropna(axis='columns', how='all'))

Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/domain_doc_ners_df.csv


,ent_phrase,ent_type,is_probe,is_probe_probability
54268,• Chin,PRODUCT,True,0.002555
62566,• Chin,PRODUCT,True,0.002555


In [192]:

mask_series = domain_doc_ners_df.ent_phrase.map(lambda x: 'cpg' in str(x).lower())# & (domain_doc_ners_df.ent_type == 'PERSON')
domain_doc_ners_df.loc[mask_series, 'is_probe'] = False
nu.save_data_frames(domain_doc_ners_df=domain_doc_ners_df)
display(domain_doc_ners_df[mask_series][analysis_columns].sort_values('is_probe_probability', ascending=False).dropna(axis='columns', how='all'))

Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/domain_doc_ners_df.csv


,ent_phrase,ent_type,is_probe,is_probe_probability
38458,Tactical Triage Protocol \n \n \nProlonged Cas...,WORK_OF_ART,False,0.002630
38433,Prolonged Casualty Care Guidelines \nCPG,WORK_OF_ART,False,0.002557
37819,Prolonged Field Care CPG,WORK_OF_ART,False,0.002542
38229,https://jts.amedd.army.mil/assets/docs/cpgs/Da...,PERSON,False,0.002536
38355,https://jts.amedd.army.mil/assets/docs/cpgs/Bu...,ORG,False,0.002534
...,...,...,...,...
37329,CPG,ORG,False,0.002466
37271,CPG,ORG,False,0.002466
37249,CPG,ORG,False,0.002466
37231,CPG,ORG,False,0.002466


In [189]:

mask_series = domain_doc_ners_df.ent_phrase.map(lambda x: bool(re.search(r'\bdecompress', str(x).lower(), re.IGNORECASE)))
# domain_doc_ners_df.loc[mask_series, 'is_probe'] = True
# nu.save_data_frames(domain_doc_ners_df=domain_doc_ners_df)
display(domain_doc_ners_df[mask_series][analysis_columns].sort_values('is_probe_probability', ascending=False).dropna(axis='columns', how='all'))

,ent_phrase,ent_type,is_probe,is_probe_probability
54301,− Decompress,ORG,True,0.003355
62599,− Decompress,ORG,True,0.003355
82643,Needle Decompress Chest,ORG,True,0.002772
82968,Needle Decompress Chest,ORG,True,0.002772


In [183]:

mask_series = domain_doc_ners_df.ent_phrase.map(lambda x: 'hemcon' in str(x).lower())# & (domain_doc_ners_df.ent_type == 'PERSON')
domain_doc_ners_df.loc[mask_series, 'is_probe'] = False
nu.save_data_frames(domain_doc_ners_df=domain_doc_ners_df)
display(domain_doc_ners_df[mask_series][analysis_columns].sort_values('is_probe_probability', ascending=False).dropna(axis='columns', how='all'))

Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/domain_doc_ners_df.csv


,ent_phrase,ent_type,is_probe,is_probe_probability
384625,HemCon ChitoGauze,PERSON,False,0.003315
384613,HemCon,PERSON,False,0.002644
384643,HemCon,ORG,False,0.002644
384660,HemCon,ORG,False,0.002644
384667,HemCon,PERSON,False,0.002644


In [180]:

mask_series = domain_doc_ners_df.ent_phrase.map(lambda x: str(x).startswith('Current Resources - Yes\nImmediate\nLikely'))
domain_doc_ners_df.loc[mask_series, 'is_probe'] = True
nu.save_data_frames(domain_doc_ners_df=domain_doc_ners_df)
display(domain_doc_ners_df[mask_series][analysis_columns].sort_values('is_probe_probability', ascending=False).dropna(axis='columns', how='all'))

Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/domain_doc_ners_df.csv


,ent_phrase,ent_type,is_probe,is_probe_probability
407401,Current Resources - Yes\nImmediate\nLikely,ORG,True,0.002547


In [178]:

mask_series = domain_doc_ners_df.ent_phrase.map(lambda x: str(x).startswith('Massive Hemorrhage\nAssess'))
domain_doc_ners_df.loc[mask_series, 'is_probe'] = True
nu.save_data_frames(domain_doc_ners_df=domain_doc_ners_df)
display(domain_doc_ners_df[mask_series][analysis_columns].sort_values('is_probe_probability', ascending=False).dropna(axis='columns', how='all'))

Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/domain_doc_ners_df.csv


,ent_phrase,ent_type,is_probe,is_probe_probability
82452,Massive Hemorrhage\nAssess,ORG,True,0.002545


In [177]:

mask_series = domain_doc_ners_df.ent_phrase.map(lambda x: str(x).startswith('Massive External \nHemorrhage'))
domain_doc_ners_df.loc[mask_series, 'is_probe'] = True
nu.save_data_frames(domain_doc_ners_df=domain_doc_ners_df)
display(domain_doc_ners_df[mask_series][analysis_columns].sort_values('is_probe_probability', ascending=False).dropna(axis='columns', how='all'))

Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/domain_doc_ners_df.csv


,ent_phrase,ent_type,is_probe,is_probe_probability
82600,Massive External \nHemorrhage,ORG,True,0.002547
82927,Massive External \nHemorrhage,ORG,True,0.002547


In [174]:

mask_series = domain_doc_ners_df.ent_phrase.map(lambda x: str(x).startswith('Immediate\nIndividual Assessment\n'))
domain_doc_ners_df.loc[mask_series, 'is_probe'] = True
nu.save_data_frames(domain_doc_ners_df=domain_doc_ners_df)
display(domain_doc_ners_df[mask_series][analysis_columns].sort_values('is_probe_probability', ascending=False).dropna(axis='columns', how='all'))

Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/domain_doc_ners_df.csv


,ent_phrase,ent_type,is_probe,is_probe_probability
407395,Immediate\nIndividual Assessment\n◻ Life Savin...,WORK_OF_ART,True,0.002547


In [172]:

mask_series = domain_doc_ners_df.ent_phrase.map(lambda x: str(x).startswith('CONTINUE TACTICAL FIELD CARE\nHemorrhage Contr'))
domain_doc_ners_df.loc[mask_series, 'is_probe'] = True
nu.save_data_frames(domain_doc_ners_df=domain_doc_ners_df)
display(domain_doc_ners_df[mask_series][analysis_columns].sort_values('is_probe_probability', ascending=False).dropna(axis='columns', how='all'))

Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/domain_doc_ners_df.csv


,ent_phrase,ent_type,is_probe,is_probe_probability
82609,CONTINUE TACTICAL FIELD CARE\nHemorrhage Contr...,ORG,True,0.002561


In [166]:

mask_series = domain_doc_ners_df.ent_phrase.map(lambda x: 'jaw' in str(x).lower())# & (domain_doc_ners_df.ent_type == 'PERSON')
domain_doc_ners_df.loc[mask_series, 'is_probe'] = True
nu.save_data_frames(domain_doc_ners_df=domain_doc_ners_df)
display(domain_doc_ners_df[mask_series][analysis_columns].sort_values('is_probe_probability', ascending=False).dropna(axis='columns', how='all'))

Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/domain_doc_ners_df.csv


,ent_phrase,ent_type,is_probe,is_probe_probability
82615,Impending Airway Obstruction\nUnconscious with...,WORK_OF_ART,True,0.002640
82944,Impending Airway Obstruction\nUnconscious with...,WORK_OF_ART,True,0.002640
82618,Chin Lift / Jaw Thrust Maneuver,PERSON,True,0.002557
82948,Chin Lift / Jaw Thrust Maneuver,PERSON,True,0.002557
36820,Jaw Thrust,PERSON,True,0.002557
362438,jaw thrust,PERSON,True,0.002557


In [153]:

mask_series = domain_doc_ners_df.ent_phrase.map(lambda x: 'nasopharyngeal' in str(x).lower())# & (domain_doc_ners_df.ent_type == 'PERSON')
domain_doc_ners_df.loc[mask_series, 'is_probe'] = True
nu.save_data_frames(domain_doc_ners_df=domain_doc_ners_df)
display(domain_doc_ners_df[mask_series][analysis_columns].sort_values('is_probe_probability', ascending=False).dropna(axis='columns', how='all'))

Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/domain_doc_ners_df.csv


,ent_phrase,ent_type,is_probe,is_probe_probability
82619,Insert Nasopharyngeal Airway,PERSON,True,0.002838
82949,Insert Nasopharyngeal Airway,PERSON,True,0.002838
54269,• Nasopharyngeal,PRODUCT,True,0.002654
54273,• Nasopharyngeal,PRODUCT,True,0.002654
62567,• Nasopharyngeal,PRODUCT,True,0.002654
62571,• Nasopharyngeal,PRODUCT,True,0.002654
82615,Impending Airway Obstruction\nUnconscious with...,WORK_OF_ART,True,0.002640
82944,Impending Airway Obstruction\nUnconscious with...,WORK_OF_ART,True,0.002640


In [122]:

mask_series = domain_doc_ners_df.ent_phrase.map(lambda x: 'celox' in str(x).lower())# & (domain_doc_ners_df.ent_type != 'PERSON')
display(domain_doc_ners_df[mask_series][analysis_columns].sort_values('is_probe_probability', ascending=False).dropna(axis='columns', how='all'))
domain_doc_ners_df.loc[mask_series, 'is_probe'] = True
nu.save_data_frames(domain_doc_ners_df=domain_doc_ners_df)

,ent_phrase,ent_type,is_probe,is_probe_probability
43620,Celox Gauze,PERSON,True,0.004086
54253,Celox Gauze,PERSON,True,0.004086
62551,Celox Gauze,PERSON,True,0.004086
83342,Celox Gauze,PERSON,True,0.004086
384627,Celox Gauze,PERSON,True,0.004086
384642,Celox \nGauze†,ORG,True,0.004086
83344,Hemostatic Celox Gauze,PERSON,True,0.003356
384670,Celox,PERSON,False,0.003302
385467,CELOX-A,ORG,False,0.003302


Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/domain_doc_ners_df.csv


In [114]:

mask_series = domain_doc_ners_df.ent_phrase.map(lambda x: 'junctional' in str(x).lower())# & (domain_doc_ners_df.ent_type != 'PERSON')
display(domain_doc_ners_df[mask_series][analysis_columns].sort_values('is_probe_probability', ascending=False).dropna(axis='columns', how='all'))
domain_doc_ners_df.loc[mask_series, 'is_probe'] = True
nu.save_data_frames(domain_doc_ners_df=domain_doc_ners_df)

,ent_phrase,ent_type,is_probe,is_probe_probability
83877,SAM-Junctional Tourniquet,ORG,True,0.003028
384696,the SAM Junctional Tourniquet,ORG,True,0.002873
83357,Tourniquet Kit Junctional Compression,ORG,True,0.002826
43636,Junctional,EVENT,False,0.002704
83352,JUNCTIONAL TOURNIQUETS & DEVICES,ORG,True,0.002621
384679,Junctional Hemorrhage,WORK_OF_ART,False,0.002596
82603,Junctional Wounds,FAC,False,0.002592
82930,Junctional Wounds,FAC,False,0.002592
385460,Control of junctional,ORG,False,0.002565
384703,the Junctional Emergency Treatment Tool,ORG,False,0.002555


Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/domain_doc_ners_df.csv


In [106]:

mask_series = domain_doc_ners_df.ent_phrase.map(lambda x: bool(re.search(r'\blimb\b', str(x).lower(), re.IGNORECASE)))# & (domain_doc_ners_df.ent_type != 'PERSON')
display(domain_doc_ners_df[mask_series][analysis_columns].sort_values('is_probe_probability', ascending=False).dropna(axis='columns', how='all'))
domain_doc_ners_df.loc[mask_series, 'is_probe'] = True
nu.save_data_frames(domain_doc_ners_df=domain_doc_ners_df)

,ent_phrase,ent_type,is_probe,is_probe_probability
82598,Limb Tourniquet,PERSON,True,0.003562
82925,Limb Tourniquet,PERSON,True,0.003562
54346,• Limb,ORG,False,0.002759
62644,• Limb,ORG,False,0.002759
82464,Convert Limb,PERSON,False,0.002613


Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/domain_doc_ners_df.csv


In [101]:

mask_series = domain_doc_ners_df.ent_phrase.map(lambda x: 'unconscious' in str(x).lower())# & (domain_doc_ners_df.ent_type != 'PERSON')
display(domain_doc_ners_df[mask_series][analysis_columns].sort_values('is_probe_probability', ascending=False).dropna(axis='columns', how='all'))
domain_doc_ners_df.loc[mask_series, 'is_probe'] = True
nu.save_data_frames(domain_doc_ners_df=domain_doc_ners_df)

,ent_phrase,ent_type,is_probe,is_probe_probability
82453,Airway Management\nUnconscious,ORG,True,0.003036
82615,Impending Airway Obstruction\nUnconscious with...,WORK_OF_ART,True,0.002640
82944,Impending Airway Obstruction\nUnconscious with...,WORK_OF_ART,True,0.002640
54613,• Unconscious casualty \n• Casualty,ORG,False,0.002559
62911,• Unconscious casualty \n• Casualty,ORG,False,0.002559


Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/domain_doc_ners_df.csv


In [87]:

mask_series = domain_doc_ners_df.ent_phrase.map(lambda x: 'combat trauma' in str(x).lower())# & (domain_doc_ners_df.ent_type != 'PERSON')
display(domain_doc_ners_df[mask_series][analysis_columns].sort_values('is_probe_probability', ascending=False).dropna(axis='columns', how='all'))
domain_doc_ners_df.loc[mask_series, 'is_probe'] = True
nu.save_data_frames(domain_doc_ners_df=domain_doc_ners_df)

,ent_phrase,ent_type,is_probe,is_probe_probability
746826,Combat Trauma,PERSON,False,0.003366
385300,Combat Trauma Lessons Learned,ORG,False,0.002751


Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/domain_doc_ners_df.csv


In [80]:

mask_series = domain_doc_ners_df.ent_phrase.map(lambda x: 'combat lifesaver' in str(x).lower())# & (domain_doc_ners_df.ent_type != 'PERSON')
display(domain_doc_ners_df[mask_series][analysis_columns].sort_values('is_probe_probability', ascending=False).dropna(axis='columns', how='all'))
domain_doc_ners_df.loc[mask_series, 'is_probe'] = True
nu.save_data_frames(domain_doc_ners_df=domain_doc_ners_df)

,ent_phrase,ent_type,is_probe,is_probe_probability
82587,Combat Lifesaver,PRODUCT,False,0.002563
82607,Combat Lifesaver,PRODUCT,False,0.002563
83186,Combat Lifesaver,PRODUCT,False,0.002563
83139,Combat Lifesaver,PRODUCT,False,0.002563
83122,Combat Lifesaver,PRODUCT,False,0.002563
83090,Combat Lifesaver,PRODUCT,False,0.002563
83050,Combat Lifesaver,PRODUCT,False,0.002563
83019,Combat Lifesaver,PRODUCT,False,0.002563
82989,Combat Lifesaver,PRODUCT,False,0.002563
82957,Combat Lifesaver,PRODUCT,False,0.002563


Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/domain_doc_ners_df.csv


In [71]:

mask_series = domain_doc_ners_df.ent_phrase.map(lambda x: 'triage' in str(x).lower()) & (domain_doc_ners_df.ent_type != 'PERSON')
domain_doc_ners_df.loc[mask_series, 'is_probe'] = True
nu.save_data_frames(domain_doc_ners_df=domain_doc_ners_df)
display(domain_doc_ners_df[mask_series][analysis_columns].sort_values('is_probe_probability', ascending=False))

Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/domain_doc_ners_df.csv


,bert_word,bert_entity,bert_score,nlp_word,nlp_tag,nlp_type,nlp_pofs,ent_phrase,ent_type,is_probe,is_probe_probability
82923,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CONTINUE TACTICAL EVACUATION CARE\nTriage,ORG,True,0.002251
36490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tactical Triage Protocol,WORK_OF_ART,True,0.002243
38456,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tactical Triage Protocol,WORK_OF_ART,True,0.002243
863547,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Performing Battlefield Triage,WORK_OF_ART,True,0.002239
407448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Morgue\nExtraction\nTriage/Treatment\nEvacuati...,ORG,True,0.002239
36484,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Triage Guiding Principles .......................,WORK_OF_ART,True,0.002239
38366,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Triage Guiding Principles,WORK_OF_ART,True,0.002239
407122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Triage Area,LOC,True,0.002238
38458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tactical Triage Protocol \n \n \nProlonged Cas...,WORK_OF_ART,True,0.002238
407067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,• Triage,ORG,True,0.002237
